In [3]:
import torch
from stable_baselines3 import DQN
import re

def extract_features_from_code(code_string):
    """Full feature extraction with 6 features as the model expects"""
    # Basic metrics
    length = len(code_string)
    num_loops = len(re.findall(r'\b(for|while)\b', code_string))
    num_conditions = len(re.findall(r'\b(if|elif|else)\b', code_string))
    num_functions = code_string.count("def ")
    num_variables = len(re.findall(r'\b\w+\s*=', code_string))
    
    # Recursion detection
    function_match = re.search(r'def\s+(\w+)\(', code_string)
    num_recursive = 0
    if function_match:
        func_name = function_match.group(1)
        num_recursive = max(len(re.findall(fr'\b{func_name}\s*\(', code_string)) - 1, 0)

    # Complexity calculation (matches original training features)
    complexity_score = (
        num_loops * 2.0 +
        num_recursive * 7.0 +
        num_variables * 0.5 +
        num_conditions * 1.5
    )

    # 6 features with proper normalization
    return [
        length / 10000,          # Feature 1: Normalized length
        num_loops / 15,          # Feature 2: Loops
        num_conditions / 10,     # Feature 3: Conditionals
        num_functions / 5,       # Feature 4: Functions
        num_variables / 20,      # Feature 5: Variables
        complexity_score / 100   # Feature 6: Complexity
    ]

def evaluate_code_efficiency(code_string):
    """Fixed observation shape evaluation"""
    try:
        # Extract features and ensure correct shape
        features = extract_features_from_code(code_string)
        obs = torch.tensor(features, dtype=torch.float32).view(1, -1)  # Shape [1, 6]
        
        # Get prediction
        action, _ = model.predict(obs.numpy(), deterministic=True)
        
        return "Efficient Code ✅" if action == 1 else "Inefficient Code ❌"
    except Exception as e:
        return f"Error: {str(e)}"

if __name__ == "__main__":
    try:
        # Load model (ensure correct path)
        model = DQN.load(r"C:\Users\Deba Prakash\Desktop\Model Traning\eco_code_ai_model.zip")
        
        # Test with Fibonacci code
        test_code = "def hello(): print(Hello World)"
        print(evaluate_code_efficiency(test_code))
    except Exception as e:
        print(f"Runtime Error: {e}")

Efficient Code ✅


In [2]:
import requests
import base64
from typing import Dict, Any
import json

def extract_github_repo_code(repo_owner: str, repo_name: str, branch: str = "main") -> Dict[str, str]:
    
    # GitHub API base URL
    base_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}"
    
    # Dictionary to store all code files
    code_files = {}
    
    def get_contents(path: str = ""):
        # API endpoint for getting repository contents
        contents_url = f"{base_url}/contents/{path}?ref={branch}"
        
        try:
            # Send GET request to GitHub API
            response = requests.get(contents_url)
            response.raise_for_status()
            contents = response.json()
            
            # If contents is a list, it's a directory
            if isinstance(contents, list):
                for item in contents:
                    if item["type"] == "file":
                        # Get file content
                        if item["encoding"] == "base64":
                            content = base64.b64decode(item["content"]).decode('utf-8')
                        else:
                            content = requests.get(item["download_url"]).text
                        code_files[item["path"]] = content
                    elif item["type"] == "dir":
                        # Recursively get contents of subdirectories
                        get_contents(item["path"])
            # If contents is a dict, it's a single file
            else:
                if contents["encoding"] == "base64":
                    content = base64.b64decode(contents["content"]).decode('utf-8')
                else:
                    content = requests.get(contents["download_url"]).text
                code_files[contents["path"]] = content
                
        except requests.exceptions.RequestException as e:
            print(f"Error accessing GitHub API: {e}")
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON response: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
    
    # Start the extraction process
    get_contents()
    return code_files

# Example usage:
if __name__ == "__main__":
    repo_owner = "username"
    repo_name = "repository"
    code_files = extract_github_repo_code(repo_owner, repo_name)
    
    # Print all extracted files
    for filepath, content in code_files.items():
        print(f"\nFile: {filepath}")
        print("Content:")
        print(content[:200] + "..." if len(content) > 200 else content)

Error accessing GitHub API: 404 Client Error: Not Found for url: https://api.github.com/repos/username/repository/contents/?ref=main
